In [1]:
import torch 
import transformers 
device = "cuda" if torch.cuda.is_available() else "cpu"
device

c:\Users\APARJOL\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'cuda'

In [2]:
import pandas as pd 

df = pd.read_csv("data/dataprocesed.csv")
labels_vect = df["Issue Type"].unique()
labels = {labels_vect[i]:i for i in range(len(labels_vect))}
df['Issue Type']=df['Issue Type'].apply(lambda x:labels[x])
tokenizer = transformers.BertTokenizer.from_pretrained("model_save",local_files_only=True)



In [3]:
import numpy as np 
MAX_LEN = 128
input_ids = []
true_labels = df["Issue Type"].to_numpy().astype(int)

orig_to_chunk ={}
orig_to_len = {}
for i in range(len(df)):
    for j in range(1,5):
        col="JuicePath."+str(j)
        if isinstance(df[col][i],str):
            encoded_sent = tokenizer.encode(df[col][i],add_special_tokens=True)
            orig_to_chunk[i]=[]
            orig_to_len[i]=[]
            if len(encoded_sent)>MAX_LEN:
                encoded_sent=encoded_sent[1:-1]
                chunk_len = MAX_LEN-2
                for k in range(0,len(encoded_sent),chunk_len):
                    tokens = encoded_sent[k:k+chunk_len]
                    chunk = [tokenizer.cls_token_id] + tokens + [tokenizer.sep_token_id]
                    input_ids.append(chunk)
                    orig_to_chunk[i].append(len(input_ids) - 1)
                    orig_to_len[i].append(len(chunk))
                    
            else:
                input_ids.append(encoded_sent)
                orig_to_chunk[i].append(len(input_ids) - 1)
                orig_to_len[i].append(len(encoded_sent))
                
print("DONE!~")
print(f"{len(input_ids)} total logfiles!")


Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors


DONE!~
14751 total logfiles!


In [4]:
from keras_preprocessing.sequence import pad_sequences

chunk_labels = [0]*len(input_ids)

test_input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, 
                               dtype="long", truncating="post", padding="post")

test_attention_masks = []

for seq in test_input_ids:
  seq_mask = [int(i>0) for i in seq]
  test_attention_masks.append(seq_mask) 

test_inputs = torch.tensor(test_input_ids)
test_masks = torch.tensor(test_attention_masks)
chunk_labels = torch.tensor(chunk_labels)


In [5]:
batch_size = 32 
from torch.utils.data import TensorDataset,DataLoader,SequentialSampler,RandomSampler

test_data = TensorDataset(test_inputs,test_masks,chunk_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data,sampler=test_sampler,batch_size=batch_size)


In [6]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("model_save",local_files_only=True)
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [7]:
def flat_accuracy(preds,labels):
    pred_flat = np.argmax(preds,axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat==labels_flat)/len(labels_flat)

import time 
import datetime

def format_time(elapsed):
    elapsed_rounded=int(round(elapsed))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [8]:
model.eval()

predictions = []

t0 = time.time()

for (step,batch) in enumerate(test_dataloader):
    batch = tuple(x.to(device) for x in batch)
    if step%10==0 and not step ==0:
        print(f"Batch {step} of {len(test_dataloader)} Elapsed : {format_time(time.time()-t0)}")
    b_input_ids , b_input_masks , b_labels = batch 
    with torch.no_grad():
        outputs = model(b_input_ids,token_type_ids =None,attention_mask=b_input_masks)
    logits = outputs[0]
    logits = logits.detach().cpu().numpy()
    predictions.append(logits)


Batch 10 of 461 Elapsed : 0:00:04
Batch 20 of 461 Elapsed : 0:00:04
Batch 30 of 461 Elapsed : 0:00:05
Batch 40 of 461 Elapsed : 0:00:06
Batch 50 of 461 Elapsed : 0:00:07
Batch 60 of 461 Elapsed : 0:00:08
Batch 70 of 461 Elapsed : 0:00:09
Batch 80 of 461 Elapsed : 0:00:09
Batch 90 of 461 Elapsed : 0:00:10
Batch 100 of 461 Elapsed : 0:00:11
Batch 110 of 461 Elapsed : 0:00:12
Batch 120 of 461 Elapsed : 0:00:13
Batch 130 of 461 Elapsed : 0:00:14
Batch 140 of 461 Elapsed : 0:00:14
Batch 150 of 461 Elapsed : 0:00:15
Batch 160 of 461 Elapsed : 0:00:16
Batch 170 of 461 Elapsed : 0:00:17
Batch 180 of 461 Elapsed : 0:00:18
Batch 190 of 461 Elapsed : 0:00:19
Batch 200 of 461 Elapsed : 0:00:19
Batch 210 of 461 Elapsed : 0:00:20
Batch 220 of 461 Elapsed : 0:00:21
Batch 230 of 461 Elapsed : 0:00:22
Batch 240 of 461 Elapsed : 0:00:23
Batch 250 of 461 Elapsed : 0:00:24
Batch 260 of 461 Elapsed : 0:00:24
Batch 270 of 461 Elapsed : 0:00:25
Batch 280 of 461 Elapsed : 0:00:26
Batch 290 of 461 Elapsed : 0:

In [9]:
predictions=np.concatenate(predictions,axis=0)
print(len(predictions))

14751


In [10]:
def compute_pred(input):
    return torch.argmax(input,dim=-1).item()
total=0
check = 0
for test_i in range(len(true_labels)):
    chunk_s_max = []
    for i in range(len(orig_to_chunk[test_i])):
        input_size = np.sum(orig_to_len[test_i])
        chunk_s_max.append(torch.softmax(torch.tensor(predictions[orig_to_chunk[test_i][i]]),dim=-1)*orig_to_len[test_i][i]/input_size)
    input_full = (np.sum(np.array(chunk_s_max),axis=0))
    input_pred = compute_pred(input_full)
    if input_pred==true_labels[test_i]:
        check+=1
    total+=1
    

print(f"Accuracy : {check/total} from {total} logs ")


C:\Users\APARJOL\AppData\Local\Temp\2\ipykernel_32948\2384863835.py:10: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  input_full = (np.sum(np.array(chunk_s_max),axis=0))
C:\Users\APARJOL\AppData\Local\Temp\2\ipykernel_32948\2384863835.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  input_full = (np.sum(np.array(chunk_s_max),axis=0))


Accuracy : 0.9963503649635036 from 1370 logs 


K-NN

In [27]:
from transformers import BertForSequenceClassification,BertTokenizer,BertConfig
config = BertConfig.from_pretrained("model_save",output_hidden_states=True)
model = BertForSequenceClassification.from_pretrained("model_save",config=config,local_files_only=True)
tokenizer = BertTokenizer.from_pretrained("model_save",local_files_only=True)
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [109]:
def line_to_embedding(tokenizer,model,line):
    chunks = []
    lens = []

    for j in range(1,5):
        col="JuicePath."+str(j)
        in_text = df[col][line]
        if isinstance(in_text,str):
            encoded_sent = tokenizer.encode(
                in_text,
                add_special_tokens = True
            )
            if len(encoded_sent)>MAX_LEN:
                encoded_sent = encoded_sent[1:-1]
                chunk_len = MAX_LEN-2
                for k in range(0,len(encoded_sent),chunk_len):
                    tokens = encoded_sent[k:k+chunk_len]
                    chunk = [tokenizer.cls_token_id] + tokens + [tokenizer.sep_token_id]
                    chunks.append(chunk)
                    lens.append(len(chunk))
            else:
                chunks.append(encoded_sent)
                lens.append(len(encoded_sent))

    

    results = pad_sequences(chunks,
                            maxlen=MAX_LEN,
                            dtype="long",
                            truncating="post",
                            padding="post"
                            )
    
    attention_masks = []
    for chunk in results:
        attn_mask = [int(token>0) for token in chunk]
        attention_masks.append(attn_mask)
    
    

    chunks = torch.tensor(results)
    attention_masks = torch.tensor(attention_masks)
  
    model.eval()

    chunks=chunks.to(device)
    attention_masks=attention_masks.to(device)
    with torch.no_grad():
        outputs= model(
            chunks,
            token_type_ids=None,
            attention_mask = attention_masks
        )
    batch_no = chunks.shape[0]
    encoded_layers = outputs[1]
    layer_i=12
    token_i=0
    vecs = []
    for batch_i in range(batch_no):
        vec = encoded_layers[layer_i][batch_i][token_i]
        vec = vec.detach().cpu().numpy()
        vecs.append(vec)
    total_len = np.sum(lens)
    vec_weights = np.array([0]*768).astype("float64")
    for i in range(len(vecs)):
        vec_weights+=vecs[i]*lens[i]/total_len
    return (vec_weights)
    


In [110]:
input_text = df['JuicePath.1'][2]
vec = line_to_embedding(tokenizer,model,2)

print(vec.shape)


(768,)


In [114]:
t0 = time.time()

embeddings = []

num_logs = len(df)
print(f"Generating Embeddings for all {num_logs} logs")

for index,row in df.iterrows():
    if index%100==0 and not index == 0:
        elapsed = format_time(time.time() - t0)
        print(f"Log {index} of {num_logs} . Elapsed : {elapsed}")
    vec = line_to_embedding(tokenizer,model,index)
    embeddings.append(vec)


Generating Embeddings for all 1370 logs
Log 100 of 1370 . Elapsed : 0:00:05
Log 200 of 1370 . Elapsed : 0:00:09
Log 300 of 1370 . Elapsed : 0:00:14
Log 400 of 1370 . Elapsed : 0:00:20
Log 500 of 1370 . Elapsed : 0:00:25
Log 600 of 1370 . Elapsed : 0:00:29
Log 700 of 1370 . Elapsed : 0:00:34
Log 800 of 1370 . Elapsed : 0:00:39
Log 900 of 1370 . Elapsed : 0:00:46
Log 1000 of 1370 . Elapsed : 0:00:52
Log 1100 of 1370 . Elapsed : 0:00:57
Log 1200 of 1370 . Elapsed : 0:01:04
Log 1300 of 1370 . Elapsed : 0:01:08


In [115]:
vecs = np.stack(embeddings)
vecs.shape

(1370, 768)

In [117]:
np.save("model_save/embeddings.npy",vecs)

In [122]:
import faiss

cpu_index = faiss.IndexFlatL2(vecs.shape[1])

t0 = time.time()
cpu_index.add(vecs)
elapsed = format_time(time.time()-t0)

print(f"Building index took {elapsed}")

Building index took 0:00:00


In [132]:
print("Input log ")
print("log # 1")

print(df['JuicePath.1'][0])

D,I = cpu_index.search(vecs[0].reshape(1,768),k=6)

for i in range(I.shape[1]):
    result_i = I[0,i]
    log = df['JuicePath.1'][result_i]
    print(f"Log # {result_i}")
    print(f"L2 distance: {D[0,i]:.2f}")
    print(log)
    print('')

Input log 
log # 1
Fut.SquadBattles.1.Test  apr                 pressing buttons  dpad right    apr                 pressing buttons  dpad right    apr                 pressing buttons  dpad right    apr                 pressing buttons  dpad right    apr                 pressing buttons  dpad right    apr                 pressing buttons  dpad right    apr                 pressing buttons  dpad right    apr                 pressing buttons  dpad right    apr                 pressing buttons  dpad right    apr                 pressing buttons  dpad right    apr                 pressing buttons  dpad right    apr                 pressing buttons  dpad right    apr                 successfully aborted the thread  thread was being aborted     apr                 the action tomainhub has timed out after            apr                 attempting to take timeout screenshot     apr                 screenshot was succesfully located  at filepath    ro filer juice   soak   sessionfiles ro dbser